In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))

## Quantum Phase Estimation (QPE)
#### In this tutorial, we will demonstrate the capabilities of qBraid Algorithms Quantum Phase Estimation Module
Begin by importing the module from qBraid Algorithms library

In [3]:
import pyqasm
from qbraid_algorithms import qpe

/Users/lukeandreesen/pyqasm/src/pyqasm/__init__.py:60: UserWarning: Importing 'pyqasm' outside a proper installation.
  warnings.warn("Importing 'pyqasm' outside a proper installation.", UserWarning)


To load a full QPE algorithm circuit as a PyQASM module, simply pass a path to the unitary U to the `load_program()` method.

In [4]:
module = qpe.load_program("QPE/unitary.qasm")

We can perform all standard [PyQASM](https://docs.qbraid.com/pyqasm/user-guide/overview) operations on the module, such as unrolling.

In [7]:
module.unroll()

Below, we display the unrolled circuits.

In [8]:
module_str = pyqasm.dumps(module)
print(module_str)

OPENQASM 3.0;
include "stdgates.inc";
qubit[4] q;
qubit[1] psi;
bit[4] b;
h q[0];
h q[1];
h q[2];
h q[3];
s psi[0];
h psi[0];
t psi[0];
cx q[0], psi[0];
tdg psi[0];
h psi[0];
sdg psi[0];
cx q[0], psi[0];
s psi[0];
h psi[0];
t psi[0];
cx q[1], psi[0];
tdg psi[0];
h psi[0];
sdg psi[0];
cx q[1], psi[0];
s psi[0];
h psi[0];
t psi[0];
cx q[1], psi[0];
tdg psi[0];
h psi[0];
sdg psi[0];
cx q[1], psi[0];
s psi[0];
h psi[0];
t psi[0];
cx q[2], psi[0];
tdg psi[0];
h psi[0];
sdg psi[0];
cx q[2], psi[0];
s psi[0];
h psi[0];
t psi[0];
cx q[2], psi[0];
tdg psi[0];
h psi[0];
sdg psi[0];
cx q[2], psi[0];
s psi[0];
h psi[0];
t psi[0];
cx q[2], psi[0];
tdg psi[0];
h psi[0];
sdg psi[0];
cx q[2], psi[0];
s psi[0];
h psi[0];
t psi[0];
cx q[2], psi[0];
tdg psi[0];
h psi[0];
sdg psi[0];
cx q[2], psi[0];
s psi[0];
h psi[0];
t psi[0];
cx q[3], psi[0];
tdg psi[0];
h psi[0];
sdg psi[0];
cx q[3], psi[0];
s psi[0];
h psi[0];
t psi[0];
cx q[3], psi[0];
tdg psi[0];
h psi[0];
sdg psi[0];
cx q[3], psi[0];
s psi[0];
h 

## Using QPE in your own OpenQASM3 program
#### qBraid algorithms makes it easy to incorporate QPE into your own OpenQASM3 circuit.
To use the QPE algorithm within your own circuit, simply pass the path to your pre-defined unitary operation to the `generate_subroutine` function. The function will generate a subroutine containing QPE for your unitary within a QASM file located in your current working directory, or any path of your choice.

In [9]:
qpe.generate_subroutine("QPE/unitary.qasm")

Subroutine 'qpe' has been added to /Users/lukeandreesen/qbraid_algos/examples/qpe.qasm


To use the subroutine in your own circuit, add `include "qpe.qasm";` to your OpenQASM file, and call the `qpe` method by passing an appropriately sized register of qubits, as well as an ancilla qubit.

In [11]:
%cat qpe.qasm

OPENQASM 3.0;
include "stdgates.inc";


gate test_gate a {
  h a;
  x a;
}

gate CU a, b {
  ctrl @ test_gate a, b;
}
 

def qpe(qubit[4] q, qubit[1] psi) {
    int n = 4;
    for int i in [0:n-1] {
        h q[i];
    }
    for int j in [0:n-1] {
        int[16] k = 1 << j;
        for int m in [0:k-1] {
            CU q[j], psi[0];
        }
    }
    // pyqasm does not currently support nested includes, so iqft defined explicitly
    // IQFT
    for int[16] i in [0:(n >> 1) - 1] {
        swap q[i], q[n - i - 1];
    }
    for int[16] i in [0:n-1] {
        int[16] target = n - i - 1;
        for int[16] j in [0:(n - target - 2)] {
            int[16] control = n - j - 1;
            int[16] k = control - target;
            cp(-2 * pi / (1 << (k + 1))) q[control], q[target];
        }
        h q[target];
    }
}